# Importing libraries 

In [ ]:
import numpy as np
import pandas as pd
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras import activations 
from tensorflow.keras.layers import MaxPool1D, MaxPool2D, Activation, Conv1D, InputLayer, Conv2D, Dense, Dropout, Flatten,MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow import keras
import re

# Generation of DNA Sequences

In [ ]:
seq_length = 200
motif_length = 20
temp_bp = ' '

sequences_char = list()

# Generating random sequences of length  200
for i in range(2500):
    sequence_char = np.array([])

    
    for j in range(765):
        num = random.randint(0, 255)
        if num <64:
            temp_bp = 'A'
        elif num < 128 and num >= 64:
            temp_bp = 'T'
        elif num < 192 and num >= 128:
            temp_bp = 'C'
        elif num <255 and num >=192:
            temp_bp = 'G'
            
        sequence_char = np.hstack((sequence_char,temp_bp))

        
    sequences_char.append(sequence_char)

    
    
sequences_str=list()
sequences_str2=list()
motif =    "ATCGATCGATCGATCGATC"
enhancer = "AAAAAAAAAAAAAAAAAAA"
len(motif)



# Function to convert   
def listToString(s):  
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 
        

    
def insert_str(string, str_to_insert, index):
    return string[:index] + str_to_insert + string[index:]

sequences_char2 = sequences_char
for i in range(len(sequences_char)):
    s = sequences_char[i]
    s = listToString(s)
    s = ''.join(s.split())
    num = random.randint(0, 200)
    ss=insert_str(s, enhancer,num)
    s=insert_str(s, motif,num)
    sequences_str.append(s)
    sequences_str2.append(ss)

  

def assign_bp_value(a):
    if a == 'A':
        return 0
    elif a == 'T':
        return 80
    elif a == 'C':
        return 170
    elif a == 'G':
        return 255

In [ ]:
print(len(sequences_str[0]))
print(len(sequences_str2[0]))


784
784


In [ ]:
len(sequences_str)

2500

In [ ]:

def get_img(seq_str):
    images = list()
    
    for i in range(2500):
        temp = np.array(list(seq_str[i]))
        temp = np.reshape(temp, (28,28))
        conv = np.zeros_like(temp, dtype='uint8')

        for j in range (28):
            for k in range (28):
                conv[j][k]=assign_bp_value(temp[j][k])
        
        images.append(conv)

    return images

# Linearly Encoded DNA Sequences

In [ ]:
images_motifs = get_img(sequences_str)
images_enhancers = get_img(sequences_str2)

# COmbining the datasets

In [ ]:
npa = np.concatenate((np.asarray(images_motifs, dtype=np.float32),np.asarray(images_enhancers, dtype=np.float32) ))
print(npa[0].shape)
print(npa[2000])

(28, 28)
[[170. 255. 170. 170.  80.  80. 255.   0.  80.   0.   0. 255. 170.  80.
   80. 170.  80.   0.  80.  80. 255. 170.   0.  80. 170.  80. 255. 255.]
 [  0. 255.   0. 255. 255.   0.  80.  80.  80.   0. 255.   0.   0. 255.
  170.  80.   0.  80.  80. 255. 170.   0.  80. 170. 255.   0.  80. 170.]
 [255.   0.  80. 170. 255.   0.  80. 170. 255.   0.  80. 170. 170. 255.
  170. 255.   0.   0.  80. 170.   0. 255. 255.   0.   0. 170. 170. 255.]
 [ 80. 255. 255.   0.  80. 255. 170.   0. 255.  80.  80. 255.  80.  80.
   80. 255. 255. 255. 255. 170.   0. 170. 255.  80.   0.   0. 255. 170.]
 [ 80. 255.   0.   0.   0.  80. 255.   0.   0.  80. 255.   0. 255.  80.
  170. 170. 255. 170.  80.  80.   0. 170.  80.  80. 170.  80.   0. 255.]
 [255.  80. 255. 255.  80.   0. 170.  80.   0.   0.   0.  80. 170.  80.
  255.   0.   0. 170.  80. 255. 170.  80. 170.   0. 255. 255.   0.   0.]
 [170.  80.  80.  80.   0.   0.   0. 255. 170.   0.  80.   0. 255. 255.
  255. 255. 255.  80.   0.  80.  80. 255. 170.  8

# CNN Model

In [ ]:
activation = 'relu'
inp = tf.keras.Input(shape=(28,28,1))

x = Conv2D(8, (3, 3), activation=activation)(inp)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation=activation)(x)
#x = MaxPooling2D((2, 2))(x)
#x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation=activation)(x)
#x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation=activation)(x)
out = Dense(1, activation='sigmoid')(x)
model = keras.Model(inp, out)

In [ ]:
model.summary()

Model: "functional_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 11, 11, 16)        1168      
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 9, 9, 32)          4640      
_________________________________________________________________
flatten_31 (Flatten)         (None, 2592)              0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 2592)            

In [ ]:
labels = np.concatenate((np.ones(2500), np.zeros(2500)))

In [ ]:
label=labels.reshape(5000,1)

In [ ]:
npa.shape, label.shape

((5000, 28, 28), (5000, 1))

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(npa, label,epochs=100, batch_size=8, validation_split=0.2)

Epoch 1/100
500/500 [==============================] - 2s 3ms/step - loss: 3.7746 - accuracy: 0.5360 - val_loss: 0.7936 - val_accuracy: 0.4420
Epoch 2/100
500/500 [==============================] - 1s 3ms/step - loss: 0.7532 - accuracy: 0.5817 - val_loss: 0.8349 - val_accuracy: 0.0000e+00
Epoch 3/100
500/500 [==============================] - 1s 3ms/step - loss: 0.6822 - accuracy: 0.6210 - val_loss: 0.9227 - val_accuracy: 0.0000e+00
Epoch 4/100
500/500 [==============================] - 1s 3ms/step - loss: 0.6706 - accuracy: 0.6240 - val_loss: 0.9362 - val_accuracy: 0.0000e+00
Epoch 5/100
500/500 [==============================] - 1s 3ms/step - loss: 0.6635 - accuracy: 0.6242 - val_loss: 1.0594 - val_accuracy: 0.0000e+00
Epoch 6/100
500/500 [==============================] - 1s 3ms/step - loss: 0.6636 - accuracy: 0.6240 - val_loss: 0.9757 - val_accuracy: 0.0000e+00
Epoch 7/100
500/500 [==============================] - 1s 3ms/step - loss: 0.6601 - accuracy: 0.6248 - val_loss: 0.8829 - 

In [ ]:
convlayer = model.layers[4]


In [ ]:
print(convlayer)

In [ ]:
weights = convlayer.get_weights()[0].squeeze()
print(weights[:,:,0,0])
#print('Convolution parameter shape: {}'.format(weights))
#print(weights)

[[ 0.20610796  0.18492539  0.19780642]
 [ 0.03051323 -0.04284444 -0.01526043]
 [-0.01570004  0.02837513  0.04252924]]


In [ ]:
a = weights[:,:,4,1]
np.interp(a, (a.min(), a.max()), (0, 255))

array([[ 81.98358717, 169.00018938, 103.90513744],
       [100.00406933,   0.        ,  85.21959786],
       [ 43.81372951, 255.        , 154.22730698]])

In [ ]:
a = weights
b = np.interp(a, (a.min(), a.max()), (0, 255))
print(np.count_nonzero(b < 65))

644


In [ ]:
print (b.shape)


(3, 3, 16, 32)


In [ ]:
3*3*16*32


4608

In [ ]:
800/4600


0.17391304347826086

In [ ]:
activation = 'relu'
inp = tf.keras.Input(shape=(28,28,1))

x = Conv2D(32, (3, 3), activation=activation)(inp)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation=activation)(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
#x = Conv2D(32, (3, 3), activation=activation)(x)
#x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation=activation)(x)
out = Dense(1, activation='sigmoid')(x)
model = keras.Model(inp, out)






activation = 'relu'
inp = tf.keras.Input(shape=(28,28,1))

x = Conv2D(8, (3, 3), activation=activation)(inp)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation=activation)(x)
x = MaxPooling2D((2, 2))(x)
#x = BatchNormalization()(x)
#x = Conv2D(32, (3, 3), activation=activation)(x)
#x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation=activation)(x)
out = Dense(1, activation='sigmoid')(x)
model = keras.Model(inp, out)

optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(npa, label,epochs=200, batch_size=8, validation_split=0.2)